In [28]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
from scipy.misc import imsave
from PIL import Image

In [7]:
prob_files=glob('/data/DW/ordinary/DRN_RCN/' +
                'DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/*/prob/*.npy')

In [8]:
prob_files

['/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_senext50_batch16/prob/DFC2020_tes_unet_senext_16_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_resnext50_batch16/prob/DFC2020_tes_unet_resnext_16_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_resnet50_batch32/prob/DFC2020_tes_unet_resnet_32_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_resnet50_batch16/prob/DFC2020_tes_unet_resnet_16_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_senet50_batch16/prob/DFC2020_tes_unet_senet_16_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_cbamnext50_batch16/prob/DFC2020_tes_unet_cbamnext_16_.npy',
 '/data/DW/ordinary/DRN_RCN/DataFusion2020_TRAD_SEMISUPERVISED/pretes_track2/unet_senext50_batch32/prob/DFC2020_tes_unet_senext_32_.npy',
 '/data/DW/ordinary/DRN_RCN/Da

In [10]:
for i in tqdm(range(len(prob_files))):
    if i==0:
        prob=np.load(prob_files[i])
    else:
        prob+=np.load(prob_files[i])

100%|██████████| 13/13 [11:51<00:00, 59.21s/it]


In [11]:
prob=prob/13.0

In [12]:
mask=prob>0.7

In [16]:
mask.shape

(5128, 8, 256, 256)

In [18]:
mask=mask.astype('uint8')

filename

In [24]:
def load_tesdata(base_dir):

    s1_dir=glob(base_dir+'s1_0'+'/'+'*.tif')
    s2_dir=[]
    lc_dir=[]
    for i in tqdm(range(len(s1_dir))):
        s1_filename=os.path.basename(s1_dir[i])
        former=s1_filename.split('_s1')[0]
        ID=s1_filename.split('_s1')[1]
        s2_dir.append(base_dir+'s2_0/'+former+'_s2'+ID)
        lc_dir.append(base_dir + 'lc_0/' + former + '_lc' + ID)

    s1_dir=np.array(s1_dir)
    s2_dir=np.array(s2_dir)
    lc_dir=np.array(lc_dir)

    return s1_dir,s2_dir,lc_dir

base_dir = '/data/PublicData/DF2020/test_track1/'
s1_dir,s2_dir,lc_dir=load_tesdata(base_dir)

100%|██████████| 5128/5128 [00:00<00:00, 248594.44it/s]


In [26]:
pre_hex_color_dict={10:'000000',0:'009900',1:'c6b044',2:'fbff13',
                    3:'b6ff05',4:'27ff87',5:'c24f44',
                    6:'a5a5a5',7:'000000',8:'f9ffa4',9:'1c0dff'}

def Hex_to_RGB(str):
    r = int(str[0:2],16)
    g = int(str[2:4],16)
    b = int(str[4:6],16)
    return [r,g,b]

def DrawResult(labels, row, col):
    num_class = 10

    X_result = np.zeros((labels.shape[0], 3))
    for i in range(num_class):
        X_result[np.where(labels == i), 0] = Hex_to_RGB(pre_hex_color_dict[i])[0]
        X_result[np.where(labels == i), 1] = Hex_to_RGB(pre_hex_color_dict[i])[1]
        X_result[np.where(labels == i), 2] = Hex_to_RGB(pre_hex_color_dict[i])[2]

    X_result = np.reshape(X_result, (row, col, 3))

    return X_result

In [27]:
net2templab={0:0,
             1:1,
             2:3,
             3:4,
             4:5,
             5:6,
             6:8,
             7:9}
outputdir='/data/PublicData/DF2020/test_track1/lc_fake/'
visdir='/data/PublicData/DF2020/test_track1/lc_fake/vis/'

In [30]:
for i in tqdm(range(5128)):
    
    tmp=mask[i]#8,256,256
    
    summ=np.sum(tmp,axis=0).squeeze()#may be have bg
    
    tmp_lab=np.argmax(tmp,axis=0).squeeze()
    
    # mapping
    
    h,w=tmp_lab.shape

    tmp_lab=tmp_lab.reshape(-1)

    label = list(map(lambda x: net2templab[x], tmp_lab))

    label = np.array(label)  # list->array

    label= label.reshape(h, w)
    
    # mask bg
    
    label[np.where(summ==0)]=7
    
    # image
    
    im = np.uint8(label + 1)
    
    # save
    
    filename=os.path.basename(lc_dir[i])
    former = filename.split('lc')[0]
    latter = filename.split('lc')[1]

    imsave(outputdir + former + 'lc' + latter, im)
    im_rgb = Image.fromarray(np.uint8(DrawResult(label.reshape(-1), 256, 256)))
    im_rgb.save(visdir + former + 'lc' + latter[:-4] + '_vis.png')

  0%|          | 0/5128 [00:00<?, ?it/s]/home/sigma_wd/anaconda3/envs/pytorch/lib/python3.5/site-packages/ipykernel_launcher.py:35: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
100%|██████████| 5128/5128 [03:47<00:00, 22.61it/s]
